## Generate downsampled txt file from annotation

Generating pandas data frame from vaa3d annotation.eswc file 

Downsamples them to the corresponding dimensions

saves as transformix compatible format (in case for sample2ara)

In progress: get endings based on the parent name


In [1]:
import os

import pandas as pd

import numpy as np

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog

In [2]:
anno_file=fdialog.askopenfile(initialdir='Z:\\', title='Select the eswc file containing the annotations').name 

xy = simpledialog.askfloat("Input", "What is the x and y resolution in um?",
                               minvalue=0.0, maxvalue=100)
z = simpledialog.askfloat("Input", "What is the z resolution in um?",
                               minvalue=0.0, maxvalue=100)
goal_xyz = simpledialog.askfloat("Input", "What do you want to downsample the resolution to '(in um)' ?",
                               minvalue=10, maxvalue=100)

outdir = fdialog.askdirectory(title='Please select the output directory')

In [3]:
anno=open(anno_file,'r')
anno_data=anno.readlines()
# heading is stored in anno_data[2], 1st line basically useless

headings=anno_data[2].split(' ')
annotations=[lines.split(' ') for lines in anno_data[3:]]

In [4]:
annotation_df=pd.DataFrame(annotations, columns=headings)

In [5]:
annotation_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,512132,3,4867.110,2506.220,838.333,1.000,512133,0,0,1,0,1\n
1,512133,3,4867.110,2506.220,838.333,1.000,-1,0,0,1,0,1\n
2,514707,3,4867.110,2506.220,838.333,1.000,-1,0,0,1,0,1\n
3,514706,3,4867.670,2507.220,838.111,1.000,514707,0,0,1,0,1\n
4,514705,3,4868.670,2508.440,838.889,1.000,514706,0,0,1,0,1\n


In [6]:
ratioxy=goal_xyz/xy
ratioz= goal_xyz/z
# calculating the downsampling factor
# based on 0.8 x 0.8 x 5um imaging resolution (x y and z)
# downsampling to match the 25um voxel allen reference template

message = (f"Sample resolution is {xy, xy, z} um in x y z. "
f"downsampling to {goal_xyz} um. "
f"dowmsample ratio is xy = {ratioxy} and z = {ratioz}.")

message

'Sample resolution is (0.8, 0.8, 5.0) um in x y z. downsampling to 25.0 um. dowmsample ratio is xy = 31.25 and z = 5.0.'

In [7]:
ds_x= pd.to_numeric(annotation_df['x'])
ds_x=ds_x/ratioxy
ds_xround=ds_x.astype(int).astype(str)
ds_y= pd.to_numeric(annotation_df['y'])
ds_y=ds_y/ratioxy
ds_yround=ds_y.astype(int).astype(str)

ds_z= pd.to_numeric(annotation_df['z'])
ds_z=ds_z/ratioz
ds_zround=ds_z.astype(int).astype(str)

# rounds by astype int, then conver to string

In [8]:
ds_coordinates= pd.DataFrame(columns=['x','y','z'])
ds_coordinates['x']=ds_xround
ds_coordinates['y']=ds_yround
ds_coordinates['z']=ds_zround
# put in to panda dataframe just in case for future use

In [9]:
q = [' '.join(x) for x in zip(ds_xround,ds_yround,ds_zround)]
# create a list where row1= x1 y1 z1 row2= x2 y2 z2 and etc 
# for writing to the text file

In [10]:
num_row=len(annotation_df.index)

out_name= outdir[3:]+ f'_{goal_xyz}voxel_trace.txt'
out_name

'AL092_25.0voxel_trace.txt'

In [11]:
f=open(outdir+'/'+out_name,'w+')
f.write('point'+'\n')
f.write(str(num_row)+'\n')

for lines in q:
    f.write(lines+'\n')

f.close()

#works, yay

In [12]:
#attempt to find endings for each branch
# the #n of the ending should not be a parent for something else

#first put these 2 columns in to list
list_n= annotation_df['#n'].tolist()
list_parent= annotation_df['parent'].tolist()

ending_list=[]
# compare each value in the #n column to the parent list, if it is not present then it should be an ending
for N,values in enumerate (list_n):
    if values not in list_parent:
        ending_list.append(N)
print(f'there are {len(ending_list)} endings')
        

there are 666 endings


In [13]:
f=open(outdir+'/endings.txt','w+')

for idx in ending_list:
    loc=q[idx]
    f.write(loc+'\n')
f.close()
    

AttributeError: 'str' object has no attribute 'type'

In [14]:
outdir

'D:/AL092'

In [15]:
outdir+'/'+out_name

'D:/AL092/AL092_25.0voxel_trace.txt'